In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,33133
2,Huelva,Confirmados PDIA 14 días,1084
3,Huelva,Tasa PDIA 14 días,"211,23604263694293"
4,Huelva,Confirmados PDIA 7 días,452
5,Huelva,Total Confirmados,33317
6,Huelva,Curados,31291
7,Huelva,Fallecidos,389


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  33133.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  9072.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 899 personas en los últimos 7 días 

Un positivo PCR cada 326 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,33133.0,452.0,1084.0,513170.0,88.079973,211.236043,94.0
Condado-Campiña,10778.0,182.0,361.0,156231.0,116.494166,231.068098,44.0
Huelva-Costa,19076.0,235.0,626.0,289548.0,81.160982,216.199041,41.0
Huelva (capital),9072.0,160.0,441.0,143837.0,111.237025,306.597051,28.0
Sierra de Huelva-Andévalo Central,2926.0,33.0,94.0,67391.0,48.967963,139.484501,9.0
Palos de la Frontera,721.0,21.0,34.0,11742.0,178.845171,289.558849,9.0
Moguer,1391.0,31.0,72.0,21867.0,141.766132,329.263273,8.0
San Juan del Puerto,579.0,21.0,47.0,9411.0,223.143130,499.415578,7.0
Gibraleón,653.0,12.0,17.0,12737.0,94.213708,133.469420,7.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Puebla de Guzmán,141.0,4.0,17.0,3092.0,129.366106,549.805951,0.0
San Juan del Puerto,579.0,21.0,47.0,9411.0,223.143130,499.415578,7.0
Beas,232.0,15.0,19.0,4341.0,345.542502,437.687169,0.0
Cerro de Andévalo (El),154.0,7.0,9.0,2327.0,300.816502,386.764074,3.0
Valverde del Camino,660.0,17.0,49.0,12750.0,133.333333,384.313725,3.0
Calañas,201.0,1.0,10.0,2768.0,36.127168,361.271676,0.0
Almendro (El),39.0,1.0,3.0,840.0,119.047619,357.142857,0.0
Moguer,1391.0,31.0,72.0,21867.0,141.766132,329.263273,8.0
Huelva (capital),9072.0,160.0,441.0,143837.0,111.237025,306.597051,28.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Calañas,201.0,1.0,10.0,2768.0,36.127168,361.271676,0.0,0.100000
Aracena,540.0,1.0,8.0,8255.0,12.113870,96.910963,1.0,0.125000
Aljaraque,1162.0,7.0,32.0,21474.0,32.597560,149.017416,1.0,0.218750
Niebla,249.0,2.0,9.0,4116.0,48.590865,218.658892,1.0,0.222222
Puebla de Guzmán,141.0,4.0,17.0,3092.0,129.366106,549.805951,0.0,0.235294
Trigueros,331.0,3.0,11.0,7862.0,38.158229,139.913508,0.0,0.272727
San Bartolomé de la Torre,307.0,2.0,7.0,3761.0,53.177346,186.120713,0.0,0.285714
Ayamonte,1335.0,4.0,13.0,21104.0,18.953753,61.599697,3.0,0.307692
Almendro (El),39.0,1.0,3.0,840.0,119.047619,357.142857,0.0,0.333333
